In [18]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

### Prepare dataset

In [19]:
df=pd.read_csv('dataset/SEntFiN-v1.1.csv')
df

,S No.,Title,Decisions,Words
0,1,SpiceJet to issue 6.4 crore warrants to promoters,"{""SpiceJet"": ""neutral""}",8
1,2,MMTC Q2 net loss at Rs 10.4 crore,"{""MMTC"": ""neutral""}",8
2,3,"Mid-cap funds can deliver more, stay put: Experts","{""Mid-cap funds"": ""positive""}",8
3,4,Mid caps now turn into market darlings,"{""Mid caps"": ""positive""}",7
4,5,"Market seeing patience, if not conviction: Pra...","{""Market"": ""neutral""}",8
...,...,...,...,...
10748,10749,"Negative on Chambal, Advanta: Mitesh Thacker","{""Chambal"": ""negative"", ""Advanta"": ""negative""}",6
10749,10750,"Small, Mid-cap stocks may emerge outperformers","{""Small"": ""positive"", ""Mid-cap stocks"": ""posit...",6
10750,10751,Rupee slips against US dollar,"{""Rupee"": ""negative"", ""US dollar"": ""neutral""}",5
10751,10752,Rupee weak against US dollar,"{""Rupee"": ""negative"", ""US dollar"": ""neutral""}",5


In [20]:
# Convert 'Decisions' column from strings to dictionaries
df['Decisions'] = df['Decisions'].apply(ast.literal_eval)
# Normalize the 'Decisions' column
decisions_df = df['Decisions'].apply(pd.Series)
decisions_df

,SpiceJet,MMTC,Mid-cap funds,Mid caps,Market,Infosys,Hudco,HOEC,Gold,Silver,...,pharmaceutical stocks,WhatsApp,BBM,FB,Indian cyclicals,Metro,"Masoor, gram",Wincor,masoor,Chambal
0,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,positive,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,neutral,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
10749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Concatenate the original DataFrame with the normalized decisions
df_expanded = pd.concat([df.drop(columns='Decisions'), decisions_df], axis=1)
df_expanded

,S No.,Title,Words,SpiceJet,MMTC,Mid-cap funds,Mid caps,Market,Infosys,Hudco,...,pharmaceutical stocks,WhatsApp,BBM,FB,Indian cyclicals,Metro,"Masoor, gram",Wincor,masoor,Chambal
0,1,SpiceJet to issue 6.4 crore warrants to promoters,8,neutral,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,MMTC Q2 net loss at Rs 10.4 crore,8,NaN,neutral,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Mid-cap funds can deliver more, stay put: Experts",8,NaN,NaN,positive,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Mid caps now turn into market darlings,7,NaN,NaN,NaN,positive,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Market seeing patience, if not conviction: Pra...",8,NaN,NaN,NaN,NaN,neutral,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10748,10749,"Negative on Chambal, Advanta: Mitesh Thacker",6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
10749,10750,"Small, Mid-cap stocks may emerge outperformers",6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10750,10751,Rupee slips against US dollar,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10751,10752,Rupee weak against US dollar,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Melt the expanded DataFrame to long format
df_long = df_expanded.melt(
    id_vars=["S No.", "Title", "Words"],
    var_name="Aspect",
    value_name="Sentiment"
).dropna(subset=["Sentiment"])
df_long

,S No.,Title,Words,Aspect,Sentiment
0,1,SpiceJet to issue 6.4 crore warrants to promoters,8,SpiceJet,neutral
82,83,SpiceJet makes top-level changes,4,SpiceJet,neutral
1509,1510,Maran makes open offer for SpiceJet,6,SpiceJet,positive
1511,1512,"SpiceJet is a buy: CK Narayan, Sharyans Resources",8,SpiceJet,positive
1513,1514,SpiceJet: Regional focus can help airline soar...,8,SpiceJet,positive
...,...,...,...,...,...
45926038,10729,"German shares outperform on Metro, Deutsche Bank",7,Metro,positive
45936795,10733,"Masoor, gram prices weaken on sluggish demand",7,"Masoor, gram",negative
45947561,10746,European shares dip early; Wincor sinks,6,Wincor,negative
45958315,10747,"Gram, masoor decline on subdued demand",6,masoor,negative


In [23]:
# Select relevant columns for classification
df_absa = df_long[["Title", "Aspect", "Sentiment"]]
df_absa

,Title,Aspect,Sentiment
0,SpiceJet to issue 6.4 crore warrants to promoters,SpiceJet,neutral
82,SpiceJet makes top-level changes,SpiceJet,neutral
1509,Maran makes open offer for SpiceJet,SpiceJet,positive
1511,"SpiceJet is a buy: CK Narayan, Sharyans Resources",SpiceJet,positive
1513,SpiceJet: Regional focus can help airline soar...,SpiceJet,positive
...,...,...,...
45926038,"German shares outperform on Metro, Deutsche Bank",Metro,positive
45936795,"Masoor, gram prices weaken on sluggish demand","Masoor, gram",negative
45947561,European shares dip early; Wincor sinks,Wincor,negative
45958315,"Gram, masoor decline on subdued demand",masoor,negative


### CNN Classifier for sentiment analysis

In [34]:
# Menggabungkan Title dan Aspect sebagai fitur teks
df_absa['Text'] = df_absa['Title']

# Encode labels
label_encoder = LabelEncoder()
df_absa['Label'] = label_encoder.fit_transform(df_absa['Sentiment'])
df_absa

,Title,Aspect,Sentiment,Text,Label
0,SpiceJet to issue 6.4 crore warrants to promoters,SpiceJet,neutral,SpiceJet to issue 6.4 crore warrants to promoters,1
82,SpiceJet makes top-level changes,SpiceJet,neutral,SpiceJet makes top-level changes,1
1509,Maran makes open offer for SpiceJet,SpiceJet,positive,Maran makes open offer for SpiceJet,2
1511,"SpiceJet is a buy: CK Narayan, Sharyans Resources",SpiceJet,positive,"SpiceJet is a buy: CK Narayan, Sharyans Resources",2
1513,SpiceJet: Regional focus can help airline soar...,SpiceJet,positive,SpiceJet: Regional focus can help airline soar...,2
...,...,...,...,...,...
45926038,"German shares outperform on Metro, Deutsche Bank",Metro,positive,"German shares outperform on Metro, Deutsche Bank",2
45936795,"Masoor, gram prices weaken on sluggish demand","Masoor, gram",negative,"Masoor, gram prices weaken on sluggish demand",0
45947561,European shares dip early; Wincor sinks,Wincor,negative,European shares dip early; Wincor sinks,0
45958315,"Gram, masoor decline on subdued demand",masoor,negative,"Gram, masoor decline on subdued demand",0


In [35]:
X = df_absa['Text']
y = df_absa['Label']
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, max_len=50):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
        self.vocab = self.build_vocab()

    def build_vocab(self):
        vocab = set()
        for text in self.texts:
            for word in text.split():
                vocab.add(word)
        word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # +1 to reserve 0 for padding
        word_to_idx['<PAD>'] = 0
        return word_to_idx

    def text_to_sequence(self, text):
        sequence = [self.vocab.get(word, 0) for word in text.split()]
        if len(sequence) < self.max_len:
            sequence.extend([0] * (self.max_len - len(sequence)))
        else:
            sequence = sequence[:self.max_len]
        return sequence

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        sequence = self.text_to_sequence(text)
        return torch.tensor(sequence), torch.tensor(label, dtype=torch.long)

In [37]:
# Create DataLoaders
train_dataset = TextDataset(X_train, y_train)
test_dataset = TextDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [38]:
# Model
class CNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes):
        super(CNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.conv1 = nn.Conv2d(1, 100, (3, embed_size))
        self.pool = nn.MaxPool2d((2, 1))
        self.fc1 = nn.Linear(100 * ((50 - 3 + 1) // 2), num_classes)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = torch.relu(self.conv1(x)).squeeze(3)
        x = self.pool(x).squeeze(2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [39]:
# Hyperparameters
vocab_size = len(train_dataset.vocab) + 1
embed_size = 50
num_classes = len(label_encoder.classes_)
learning_rate = 0.001
num_epochs = 5

In [43]:
# run with cuda 12
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [44]:
torch.cuda.is_available()

False

In [40]:
# Initialize model, loss function, and optimizer
model = CNNModel(vocab_size, embed_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [41]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    for texts, labels in train_loader:
        outputs = model(texts)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 1.0456


KeyboardInterrupt: 

In [ ]:
# Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{report}')
